# Create adatom defects in a slab material

Create an adatom by specifying the chemical element, approximate position on surface and distance z, which will be resolved to:
- the equidistant position between the closes atoms on the surface according to Voronoi tesselation, 
- or the crystal site of the next layer that is closest to specified position.

<h2 style="color:green">Usage</h2>

1. Make sure to select Input Materials (in the outer runtime) before running the notebook.
1. Set notebook parameters in cell 1.1. below (or use the default values).
1. Set defects parameters in cell 2.1. (or use default).
1. Click “Run” > “Run All” to run all cells. 
1. Wait for the run to complete (depending on the parameters can take a few min). 
1. Scroll down to view results. 

## Summary
1. Prepare the Environment: Set up the notebook and install packages, preview the input materials
1. Create the Defect: Add an adatom defect to the slab material
2. Visualize the Defect: Visualize the defect structure

## Notes

1. For more information, see [Introduction](Introduction.ipynb)
<!-- # TODO: use a hashtag-based anchor link to interface creation documention above -->


## 1. Prepare the Environment
### 1.1. Set up defect parameters 

In [2]:
from mat3ra.made.tools.analyze import get_atomic_coordinates_extremum

DEFECT_TYPE = "island"
APPROXIMATE_POSITION_ON_SURFACE = [0.5, 0.5]  # Position of the defect in crystal coordinates
DISTANCE_Z = 5.0  # Distance of the defect from the surface in Angstrom
CHEMICAL_ELEMENT = "Ni"  # Element to be placed at the site 
MILLER_INDICES = (1, 1, 1)  # Miller indices of the surface
SLAB_THICKNESS = 5  # Thickness of the slab in unit cells
VACUUM = 6  # Vacuum thickness in Angstrom
SUPERCELL_MATRIX = [[5, 0, 0], [0, 5, 0], [0, 0, 1]]

### 1.2. Install Packages
The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).

In [3]:
import sys

if sys.platform == "emscripten":
    import micropip

    await micropip.install('mat3ra-api-examples', deps=False)
    from utils.jupyterlite import install_packages

    await install_packages("create_point_defect.ipynb", "../../config.yml")

### 1.3. Get input materials
Materials are loaded with `get_data()`. The first material is assigned as substrate and the second as film.

In [4]:
from mat3ra.made.material import Material
from utils.jupyterlite import get_data

# Get the list of input materials and load them into `materials_in` variable
get_data("materials_in", globals())
materials = list(map(Material, globals()["materials_in"]))

0: Data from 0-Ni has been read successfully.
1: Data from 1-Graphene has been read successfully.
2: Data from 10-Al2O3, Sapphire, RHL (R-3c) 3D (Bulk), mp-1143 (1) has been read successfully.
3: Data from 11-ZnO, Zinc Oxide, HEX (P6_3mc) 3D (Bulk), mp-2133 has been read successfully.
4: Data from 12-Cd4 Te4 has been read successfully.
5: Data from 13-Si4 C4 has been read successfully.
6: Data from 14-GaN, Gallium Nitride, HEX (P6_3mc) 3D (Bulk), mp-804 has been read successfully.
7: Data from 4-Te2Mo has been read successfully.
8: Data from 5-HfO2 has been read successfully.
9: Data from 6-Ni4(110), termination Ni_Pmmm_2, Slab, Terrace, 1 steps, [2 0 0] has been read successfully.
10: Data from 7-Ag4 has been read successfully.
11: Data from 8-Si, Silicene, HEX (P-3m1) 2D (Monolayer), 2dm-5934 has been read successfully.
12: Data from 9-GaAs, Gallium Arsenide, FCC (F-43m) 3D (Bulk), mp-2534 has been read successfully.
13: Data from C2(001)-C2(001), Interface, Strain 0.000pct has been 

### 1.4. Create and preview Slab

In [5]:
from mat3ra.made.tools.build.slab import SlabConfiguration, get_terminations, create_slab
from utils.visualize import visualize_materials as visualize
from mat3ra.made.tools.build.defect.builders import CrystalSiteAdatomSlabDefectBuilder

material = materials[8]
slab_config = SlabConfiguration(
    bulk=material,
    miller_indices=MILLER_INDICES,
    thickness=SLAB_THICKNESS,
    vacuum=VACUUM,
    use_orthogonal_z=True,
    xy_supercell_matrix=SUPERCELL_MATRIX
)
termination = get_terminations(slab_config)[1]
slab = create_slab(slab_config, termination)
print(slab.metadata.get("build").get("configuration").get("type"))
new_slab = CrystalSiteAdatomSlabDefectBuilder().create_material_with_additional_layers(slab, 2)
visualize([{"material": slab, "rotation": "0x"}, {"material": slab, "rotation": "-90x"}], repetitions=[1, 1, 1])
visualize([{"material": new_slab, "rotation": "0x"}, {"material": new_slab, "rotation": "-90x"}], repetitions=[1, 1, 1])
print(slab.metadata.get("build").get("configuration").get("type"))



SlabConfiguration


GridBox(children=(VBox(children=(Label(value='Hf500O1000 - Material - rotation: 0x', layout=Layout(align_self=…

GridBox(children=(VBox(children=(Label(value='Hf700O1400 - Material - rotation: 0x', layout=Layout(align_self=…

SlabConfiguration


## 2. Create the Defect
### 2.1. Set adatom parameters

In [8]:
from mat3ra.made.tools.build.defect.builders import IslandSlabDefectBuilder
from mat3ra.made.tools.build.defect.configuration import IslandSlabDefectConfiguration
from mat3ra.made.tools.utils import coordinate as CoordinateCondition
from mat3ra.made.tools.modify import filter_by_box
condition = CoordinateCondition.CylinderCoordinateCondition(center_position=[0.25, 0.5], radius=0.25, min_z=0, max_z=1)
island_config = IslandSlabDefectConfiguration(
    crystal=slab,
    defect_type="island",
    condition=condition,
    number_of_added_layers=3,
)

slab_with_island = IslandSlabDefectBuilder().get_material(configuration=island_config)
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": "Material with island defect"}])
visualize([{"material": slab, "title": "Original material"},
           {"material": slab_with_island, "title": "Material with island defect"}], rotation="-90x")
slab_with_island.metadata.get("build").get("configuration")


slab_with_island = filter_by_box(slab_with_island, [0.5, 0.5, 0.25], [0.75, 0.75, 1], invert_selection=True)
slab_with_island.name = slab_with_island.name + " Island"
visualize([{"material": slab_with_island, "title": "Material with island defect"}], rotation="-40x")

GridBox(children=(VBox(children=(Label(value='Hf500O1000 - Original material - rotation: 0x,0y,0z', layout=Lay…

GridBox(children=(VBox(children=(Label(value='Hf500O1000 - Original material - rotation: -90x', layout=Layout(…

GridBox(children=(VBox(children=(Label(value='Hf546O1090 - Material with island defect - rotation: -40x', layo…

### 2.2. Create the adatom

In [7]:

from mat3ra.made.tools.modify import filter_material_by_ids,filter_by_box
import numpy as np
from typing import Literal, List


def get_surface_atoms_indices(
    material: Material,
    surface: Literal["top", "bottom", "both"] = "top",
    n: int = 15,
    m: int = 15
) -> List[int]:
    """
    Get the indices of atoms on the surface of the material along the Z axis.

    Args:
        material (Material): Material object.
        surface (str): "top", "bottom", or "both".
        n (int): Number of divisions along the X axis.
        m (int): Number of divisions along the Y axis.

    Returns:
        List[int]: List of indices of atoms on the surface.
    """
    surface_atoms_indices = []

    # Get columns of atoms with grid n by m:
    for i in range(n):
        for j in range(m):
            # Define the bounding box for the current grid cell
            x_min, x_max = i / n, (i + 1) / n
            y_min, y_max = j / m, (j + 1) / m

            # Filter atoms within this grid cell
            atoms_column = filter_by_box(material, [x_min, y_min, 0], [x_max, y_max, 1])
           

            if len(atoms_column.basis.coordinates.values) == 0:
                continue  # Skip empty columns


            # Extract the z-coordinates
            z_values = [coord[2] for coord in atoms_column.basis.coordinates.values]

            # Find the top and bottom atoms by Z-coordinate
            top_atom_idx = np.argmax(z_values)
            bottom_atom_idx = np.argmin(z_values)

            # Map these to the corresponding IDs
            top_atom_id = atoms_column.basis.coordinates.ids[top_atom_idx]
            bottom_atom_id = atoms_column.basis.coordinates.ids[bottom_atom_idx]

            # Add the appropriate IDs to the surface_atoms_indices list
            if surface == "top" or surface == "both":
                surface_atoms_indices.append(top_atom_id)
            if surface == "bottom" or surface == "both":
                surface_atoms_indices.append(bottom_atom_id)

    # Remove duplicates by converting the list to a set and back to a list
    surface_atoms_indices = list(set(surface_atoms_indices))

    return surface_atoms_indices

ids = get_surface_atoms_indices(slab_with_island, surface="top")
top_atoms = filter_material_by_ids(slab_with_island, ids)

visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": top_atoms, "title": "Surface Atoms only"}], rotation="-90x")
visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": top_atoms, "title": "Surface Atoms only"}], rotation="0x")

KeyboardInterrupt: 

## 3. Visualize the Slabs with Adatom

## 4. Pass data to the outside runtime

In [7]:

from mat3ra.made.tools.build.defect import create_slab_defect, TerraceSlabDefectBuilder
from mat3ra.made.tools.build.defect.configuration import TerraceSlabDefectConfiguration
from scipy.spatial import cKDTree

def get_surface_atoms_indices(material: Material, distance_threshold: float = 2.5, depth: float = 5) -> List[int]:
    """
    Identify exposed atoms on the top surface of the material.

    Args:
        material (Material): Material object to get surface atoms from.
        distance_threshold (float): Distance threshold to determine if an atom is considered "covered".
        depth (float): Depth from the top surface to look for exposed atoms.

    Returns:
        List[int]: List of indices of exposed top surface atoms.
    """
    material.to_cartesian()
    coordinates = np.array(material.basis.coordinates.values)
    ids = material.basis.coordinates.ids
    kd_tree = cKDTree(coordinates)
    z_max = np.max(coordinates[:, 2])

    exposed_atoms_indices = []
    for idx, (x, y, z) in enumerate(coordinates):
        if z >= z_max - depth:
            neighbors_above = kd_tree.query_ball_point([x, y, z + distance_threshold], r=distance_threshold)

            if not any(coordinates[n][2] > z for n in neighbors_above):
                exposed_atoms_indices.append(ids[idx])
    material.to_crystal()
    return exposed_atoms_indices


configuration = TerraceSlabDefectConfiguration(
    crystal=slab,
)
builder = TerraceSlabDefectBuilder()
terrace = create_slab_defect(configuration, builder)

exposed_top_surface_atoms = get_surface_atoms_indices(slab_with_island, distance_threshold=2.5, depth=15)
exposed_atoms = filter_material_by_ids(slab_with_island, exposed_top_surface_atoms)
exposed_atoms.name = exposed_atoms.name + " Surface Atoms"

visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": exposed_atoms, "title": "Exposed Atoms only"}], rotation="-90x")
visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": exposed_atoms, "title": "Exposed Atoms only"}], rotation="0x")

GridBox(children=(VBox(children=(Label(value='Hf528O1037 - Material with island defect - rotation: -90x', layo…

GridBox(children=(VBox(children=(Label(value='Hf528O1037 - Material with island defect - rotation: 0x', layout…

In [8]:
from utils.jupyterlite import set_data
slab_with_island.name = slab_with_island.name + " Island"
exposed_atoms.name = exposed_atoms.name + " Surface Atoms"
set_data("materials_out", [slab_with_island.to_json(), exposed_atoms.to_json()])
len(slab_with_island.basis.elements.ids)

Data for materials_out written to uploads/Hf4O8(111), termination O2_P2:m_1, Slab Island Island.json
Data for materials_out written to uploads/Hf4O8(111), termination O2_P2:m_1, Slab Island Surface Atoms Surface Atoms.json


1565

In [14]:
def passivate_top(material, element, bond_length, depth):
    """
    Passivate the top surface of the material with the specified element.

    Args:
        material (Material): Material object to passivate.
        element (str): Element to use for passivation.
        bond_length (float): Bond length to use for the passivation.
        depth (float): Depth from the top surface to passivate.

    Returns:
        Material: Material object with the top surface passivated.
    """
    new_material = material.clone()
    top_atom_indices = get_surface_atoms_indices(material, distance_threshold=2.2, depth=depth)
    top_atoms = filter_material_by_ids(material, top_atom_indices)
    new_material.to_cartesian()
    top_atoms.to_cartesian()
    top_atoms_coordinates = top_atoms.basis.coordinates.values
    passivant_coordinates = np.array(top_atoms_coordinates) + [0, 0, bond_length]
    
    for coord in passivant_coordinates:
        new_material.add_atom(element, coord)
    new_material.to_crystal()
    return new_material

passivated_slab = passivate_top(slab_with_island, "H", 1.0, 15)
passivated_slab.name = passivated_slab.name + " Passivated"
visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": passivated_slab, "title": "Passivated Material"}], rotation="-90x")
visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": passivated_slab, "title": "Passivated Material"}], rotation="0x")

set_data("materials_out", [ passivated_slab.to_json()])

GridBox(children=(VBox(children=(Label(value='Hf528O1037 - Material with island defect - rotation: -90x', layo…

GridBox(children=(VBox(children=(Label(value='Hf528O1037 - Material with island defect - rotation: 0x', layout…

Data for materials_out written to uploads/Hf4O8(111), termination O2_P2:m_1, Slab Island Island Passivated.json


In [10]:
len(passivated_slab.basis.elements.values)

1708

In [11]:
from mat3ra.made.tools.modify import passivate_material, passivate_surface

passivated_surface = passivate_surface(slab_with_island, passivant="H", default_bond_length=1.0)
visualize([{"material": slab_with_island, "title": "Material with island defect"}, {"material": passivated_surface, "title": "Passivated Material"}], rotation="-90x")

GridBox(children=(VBox(children=(Label(value='Hf528O1037 - Material with island defect - rotation: -90x', layo…

In [21]:
import numpy as np
from scipy.optimize import minimize

def find_commensurate_lattice(T, a1, a2, tol=1e-6):
    """
    Finds commensurate lattice vectors for two 2D arbitrary bases.
    
    Parameters:
    - T: 2x2 numpy array, the transformation matrix relating the two bases.
    - a1, a2: numpy arrays, the vectors of the first basis.
    - tol: float, tolerance for finding integer solutions.
    
    Returns:
    - (m1, m2, n1, n2): tuple of integers that satisfy the commensurability condition.
    """
    
    def objective(vars):
        m1, m2, n1, n2 = vars
        return m1**2 + m2**2 + n1**2 + n2**2

    def constraint(vars):
        m1, m2, n1, n2 = vars
        lhs = m1 * a1 + m2 * a2
        rhs = n1 * (T @ a1) + n2 * (T @ a2)
        return np.linalg.norm(lhs - rhs)
    
    # Initial guess for m1, m2, n1, n2
    initial_guess = [1, 0, 1, 0]
    
    # Constraints dictionary
    constraints = {'type': 'eq', 'fun': constraint}
    
    # Solve the optimization problem
    result = minimize(objective, initial_guess, constraints=constraints, method='SLSQP', options={'ftol': tol})
    
    if result.success:
        m1, m2, n1, n2 = np.round(result.x).astype(int)
        return m1, m2, n1, n2
    else:
        raise ValueError("No commensurate lattice found within the given tolerance.")

# Example usage:
a1 = np.array([1, 0])
a2 = np.array([-0.5, np.sqrt(3) / 2])
angle = np.pi / 3
T = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
m1, m2, n1, n2 = find_commensurate_lattice(T, a1, a2, tol=1e-1)
print(f"Commensurate lattice vectors: m1={m1}, m2={m2}, n1={n1}, n2={n2}")


Commensurate lattice vectors: m1=0, m2=0, n1=0, n2=0


In [14]:
from mat3ra.made.tools.build.defect import create_slab_defect, IslandSlabDefectBuilder
clean_material = Material.create(Material.default_config)
slab_111_config = SlabConfiguration(
    bulk=clean_material,
    miller_indices=(0,0, 1),
    thickness=3,
    vacuum=3,
    xy_supercell_matrix=[[2, 0], [0, 2]],
    use_orthogonal_z=True,
    make_primitive=False
)
t_111 = get_terminations(slab_111_config)[0]
SLAB_111 = create_slab(slab_111_config, t_111)

def test_create_island():
    condition = CoordinateCondition.CylinderCoordinateCondition(
        center_position=[0.5, 0.5], radius=0.15, min_z=0, max_z=1
    )
    island_config = IslandSlabDefectConfiguration(
        crystal=SLAB_111,
        defect_type="island",
        condition=condition,
        number_of_added_layers=1,
    )

    defect = create_slab_defect(configuration=island_config, builder=IslandSlabDefectBuilder())
    visualize([{"material": SLAB_111, "title": "Original material"}, {"material": defect, "title": "Material with island defect"}])
    visualize([{"material": SLAB_111, "title": "Original material"}, {"material": defect, "title": "Material with island defect"}], rotation="-90x")

    # Only 2 atoms in the island were added for this configuration
    NUMBER_OF_ATOMS_IN_ISLAND = 2
    

    
test_create_island()


GridBox(children=(VBox(children=(Label(value='Si96 - Original material - rotation: 0x,0y,0z', layout=Layout(al…

GridBox(children=(VBox(children=(Label(value='Si96 - Original material - rotation: -90x', layout=Layout(align_…

In [12]:
gh_test_slab = {'name': 'Si8(111), termination Si_P6/mmm_4, Slab', 'basis': {'elements': [{'id': 0, 'value': 'Si'}, {'id': 1, 'value': 'Si'}, {'id': 2, 'value': 'Si'}, {'id': 3, 'value': 'Si'}, {'id': 4, 'value': 'Si'}, {'id': 5, 'value': 'Si'}, {'id': 6, 'value': 'Si'}, {'id': 7, 'value': 'Si'}, {'id': 8, 'value': 'Si'}, {'id': 9, 'value': 'Si'}, {'id': 10, 'value': 'Si'}, {'id': 11, 'value': 'Si'}, {'id': 12, 'value': 'Si'}, {'id': 13, 'value': 'Si'}, {'id': 14, 'value': 'Si'}, {'id': 15, 'value': 'Si'}, {'id': 16, 'value': 'Si'}, {'id': 17, 'value': 'Si'}, {'id': 18, 'value': 'Si'}, {'id': 19, 'value': 'Si'}, {'id': 20, 'value': 'Si'}, {'id': 21, 'value': 'Si'}, {'id': 22, 'value': 'Si'}, {'id': 23, 'value': 'Si'}, {'id': 24, 'value': 'Si'}, {'id': 25, 'value': 'Si'}, {'id': 26, 'value': 'Si'}, {'id': 27, 'value': 'Si'}, {'id': 28, 'value': 'Si'}, {'id': 29, 'value': 'Si'}, {'id': 30, 'value': 'Si'}, {'id': 31, 'value': 'Si'}], 'coordinates': [{'id': 0, 'value': [0.583333333, 0.958333333, 0.0125]}, {'id': 1, 'value': [0.583333333, 0.458333333, 0.0875]}, {'id': 2, 'value': [0.083333333, 0.458333333, 0.0125]}, {'id': 3, 'value': [0.083333333, 0.958333333, 0.0875]}, {'id': 4, 'value': [0.583333333, 0.458333333, 0.0125]}, {'id': 5, 'value': [0.583333333, 0.958333333, 0.0875]}, {'id': 6, 'value': [0.083333333, 0.958333333, 0.0125]}, {'id': 7, 'value': [0.083333333, 0.458333333, 0.0875]}, {'id': 8, 'value': [0.25, 0.625, 0.1125]}, {'id': 9, 'value': [0.25, 0.125, 0.1875]}, {'id': 10, 'value': [0.75, 0.125, 0.1125]}, {'id': 11, 'value': [0.75, 0.625, 0.1875]}, {'id': 12, 'value': [0.25, 0.125, 0.1125]}, {'id': 13, 'value': [0.25, 0.625, 0.1875]}, {'id': 14, 'value': [0.75, 0.625, 0.1125]}, {'id': 15, 'value': [0.75, 0.125, 0.1875]}, {'id': 16, 'value': [0.916666667, 0.291666667, 0.2125]}, {'id': 17, 'value': [0.916666667, 0.791666667, 0.2875]}, {'id': 18, 'value': [0.416666667, 0.791666667, 0.2125]}, {'id': 19, 'value': [0.416666667, 0.291666667, 0.2875]}, {'id': 20, 'value': [0.916666667, 0.791666667, 0.2125]}, {'id': 21, 'value': [0.916666667, 0.291666667, 0.2875]}, {'id': 22, 'value': [0.416666667, 0.291666667, 0.2125]}, {'id': 23, 'value': [0.416666667, 0.791666667, 0.2875]}, {'id': 24, 'value': [0.583333333, 0.958333333, 0.3125]}, {'id': 25, 'value': [0.583333333, 0.458333333, 0.3875]}, {'id': 26, 'value': [0.083333333, 0.458333333, 0.3125]}, {'id': 27, 'value': [0.083333333, 0.958333333, 0.3875]}, {'id': 28, 'value': [0.583333333, 0.458333333, 0.3125]}, {'id': 29, 'value': [0.583333333, 0.958333333, 0.3875]}, {'id': 30, 'value': [0.083333333, 0.958333333, 0.3125]}, {'id': 31, 'value': [0.083333333, 0.458333333, 0.3875]}], 'units': 'crystal', 'cell': [[7.734, 0.0, 0.0], [3.867, 6.697840473, 0.0], [0.0, 0.0, 31.573922786]], 'constraints': [], 'labels': []}, 'lattice': {'a': 7.734, 'b': 7.734, 'c': 31.573922786, 'alpha': 90.0, 'beta': 90.0, 'gamma': 60.0, 'units': {'length': 'angstrom', 'angle': 'degree'}, 'type': 'TRI', 'vectors': {'a': [7.734, 0.0, 0.0], 'b': [3.867, 6.697840473, 0.0], 'c': [0.0, 0.0, 31.573922786], 'alat': 1, 'units': 'angstrom'}}, 'isNonPeriodic': False, '_id': '', 'metadata': {'boundaryConditions': {'type': 'pbc', 'offset': 0}, 'build': {'termination': 'Si_P6/mmm_4', 'configuration': {'type': 'SlabConfiguration', 'bulk': {'name': 'Si8', 'basis': {'elements': [{'id': 0, 'value': 'Si'}, {'id': 1, 'value': 'Si'}, {'id': 2, 'value': 'Si'}, {'id': 3, 'value': 'Si'}, {'id': 4, 'value': 'Si'}, {'id': 5, 'value': 'Si'}, {'id': 6, 'value': 'Si'}, {'id': 7, 'value': 'Si'}], 'coordinates': [{'id': 0, 'value': [0.5, 0.0, 0.0]}, {'id': 1, 'value': [0.25, 0.25, 0.75]}, {'id': 2, 'value': [0.5, 0.5, 0.5]}, {'id': 3, 'value': [0.25, 0.75, 0.25]}, {'id': 4, 'value': [0.0, 0.0, 0.5]}, {'id': 5, 'value': [0.75, 0.25, 0.25]}, {'id': 6, 'value': [0.0, 0.5, 0.0]}, {'id': 7, 'value': [0.75, 0.75, 0.75]}], 'units': 'crystal', 'cell': [[5.468763846, 0.0, 0.0], [0.0, 5.468763846, 0.0], [0.0, 0.0, 5.468763846]], 'constraints': [], 'labels': []}, 'lattice': {'a': 5.468763846, 'b': 5.468763846, 'c': 5.468763846, 'alpha': 90.0, 'beta': 90.0, 'gamma': 90.0, 'units': {'length': 'angstrom', 'angle': 'degree'}, 'type': 'TRI', 'vectors': {'a': [5.468763846, 0.0, 0.0], 'b': [0.0, 5.468763846, 0.0], 'c': [0.0, 0.0, 5.468763846], 'alat': 1, 'units': 'angstrom'}}, 'isNonPeriodic': False, '_id': '', 'metadata': {'boundaryConditions': {'type': 'pbc', 'offset': 0}}, 'isUpdated': True}, 'miller_indices': (1, 1, 1), 'thickness': 4, 'vacuum': 6, 'xy_supercell_matrix': [[1, 0], [0, 1]], 'use_conventional_cell': True, 'use_orthogonal_z': True, 'make_primitive': False}}}, 'isUpdated': True}

gh_test_material = {'name': 'Si8(111), termination Si_P6/mmm_4, Slab', 'lattice': {'a': 7.734, 'b': 7.734, 'c': 31.573922786, 'alpha': 90.0, 'beta': 90.0, 'gamma': 60.0, 'units': {'length': 'angstrom', 'angle': 'degree'}, 'type': 'TRI', 'vectors': [[7.734, 0.0, 4.735709172302815e-16], [3.8670000000000018, 6.697840472868847, 4.735709172302815e-16], [0.0, 0.0, 31.573922786]]}, 'basis': {'elements': [{'id': 0, 'value': 'Si'}, {'id': 1, 'value': 'Si'}, {'id': 2, 'value': 'Si'}, {'id': 3, 'value': 'Si'}, {'id': 4, 'value': 'Si'}, {'id': 5, 'value': 'Si'}, {'id': 6, 'value': 'Si'}, {'id': 7, 'value': 'Si'}, {'id': 8, 'value': 'Si'}, {'id': 9, 'value': 'Si'}, {'id': 10, 'value': 'Si'}, {'id': 11, 'value': 'Si'}, {'id': 12, 'value': 'Si'}, {'id': 13, 'value': 'Si'}, {'id': 14, 'value': 'Si'}, {'id': 15, 'value': 'Si'}, {'id': 16, 'value': 'Si'}, {'id': 17, 'value': 'Si'}, {'id': 18, 'value': 'Si'}, {'id': 19, 'value': 'Si'}, {'id': 20, 'value': 'Si'}, {'id': 21, 'value': 'Si'}, {'id': 22, 'value': 'Si'}, {'id': 23, 'value': 'Si'}, {'id': 24, 'value': 'Si'}, {'id': 25, 'value': 'Si'}, {'id': 26, 'value': 'Si'}, {'id': 27, 'value': 'Si'}, {'id': 28, 'value': 'Si'}, {'id': 29, 'value': 'Si'}, {'id': 30, 'value': 'Si'}, {'id': 31, 'value': 'Si'}], 'coordinates': [{'id': 0, 'value': [0.583333333, 0.958333333, 0.0125]}, {'id': 1, 'value': [0.583333333, 0.458333333, 0.0875]}, {'id': 2, 'value': [0.083333333, 0.458333333, 0.0125]}, {'id': 3, 'value': [0.083333333, 0.958333333, 0.0875]}, {'id': 4, 'value': [0.583333333, 0.458333333, 0.0125]}, {'id': 5, 'value': [0.583333333, 0.958333333, 0.0875]}, {'id': 6, 'value': [0.083333333, 0.958333333, 0.0125]}, {'id': 7, 'value': [0.083333333, 0.458333333, 0.0875]}, {'id': 8, 'value': [0.25, 0.625, 0.1125]}, {'id': 9, 'value': [0.25, 0.125, 0.1875]}, {'id': 10, 'value': [0.75, 0.125, 0.1125]}, {'id': 11, 'value': [0.75, 0.625, 0.1875]}, {'id': 12, 'value': [0.25, 0.125, 0.1125]}, {'id': 13, 'value': [0.25, 0.625, 0.1875]}, {'id': 14, 'value': [0.75, 0.625, 0.1125]}, {'id': 15, 'value': [0.75, 0.125, 0.1875]}, {'id': 16, 'value': [0.916666667, 0.291666667, 0.2125]}, {'id': 17, 'value': [0.916666667, 0.791666667, 0.2875]}, {'id': 18, 'value': [0.416666667, 0.791666667, 0.2125]}, {'id': 19, 'value': [0.416666667, 0.291666667, 0.2875]}, {'id': 20, 'value': [0.916666667, 0.791666667, 0.2125]}, {'id': 21, 'value': [0.916666667, 0.291666667, 0.2875]}, {'id': 22, 'value': [0.416666667, 0.291666667, 0.2125]}, {'id': 23, 'value': [0.416666667, 0.791666667, 0.2875]}, {'id': 24, 'value': [0.583333333, 0.958333333, 0.3125]}, {'id': 25, 'value': [0.583333333, 0.458333333, 0.3875]}, {'id': 26, 'value': [0.083333333, 0.458333333, 0.3125]}, {'id': 27, 'value': [0.083333333, 0.958333333, 0.3875]}, {'id': 28, 'value': [0.583333333, 0.458333333, 0.3125]}, {'id': 29, 'value': [0.583333333, 0.958333333, 0.3875]}, {'id': 30, 'value': [0.083333333, 0.958333333, 0.3125]}, {'id': 31, 'value': [0.083333333, 0.458333333, 0.3875]}], 'units': 'crystal', 'cell': [[7.734, 0.0, 0.0], [3.867, 6.69784, 0.0], [0.0, 0.0, 31.573923]], 'labels': []}, 'metadata': {'boundaryConditions': {'type': 'pbc', 'offset': 0}, 'build': {'termination': 'Si_P6/mmm_4', 'configuration': {'type': 'IslandSlabDefectConfiguration', 'crystal': {'name': 'Si8(111), termination Si_P6/mmm_4, Slab', 'basis': {'elements': [{'id': 0, 'value': 'Si'}, {'id': 1, 'value': 'Si'}, {'id': 2, 'value': 'Si'}, {'id': 3, 'value': 'Si'}, {'id': 4, 'value': 'Si'}, {'id': 5, 'value': 'Si'}, {'id': 6, 'value': 'Si'}, {'id': 7, 'value': 'Si'}, {'id': 8, 'value': 'Si'}, {'id': 9, 'value': 'Si'}, {'id': 10, 'value': 'Si'}, {'id': 11, 'value': 'Si'}, {'id': 12, 'value': 'Si'}, {'id': 13, 'value': 'Si'}, {'id': 14, 'value': 'Si'}, {'id': 15, 'value': 'Si'}, {'id': 16, 'value': 'Si'}, {'id': 17, 'value': 'Si'}, {'id': 18, 'value': 'Si'}, {'id': 19, 'value': 'Si'}, {'id': 20, 'value': 'Si'}, {'id': 21, 'value': 'Si'}, {'id': 22, 'value': 'Si'}, {'id': 23, 'value': 'Si'}, {'id': 24, 'value': 'Si'}, {'id': 25, 'value': 'Si'}, {'id': 26, 'value': 'Si'}, {'id': 27, 'value': 'Si'}, {'id': 28, 'value': 'Si'}, {'id': 29, 'value': 'Si'}, {'id': 30, 'value': 'Si'}, {'id': 31, 'value': 'Si'}], 'coordinates': [{'id': 0, 'value': [0.583333333, 0.958333333, 0.0125]}, {'id': 1, 'value': [0.583333333, 0.458333333, 0.0875]}, {'id': 2, 'value': [0.083333333, 0.458333333, 0.0125]}, {'id': 3, 'value': [0.083333333, 0.958333333, 0.0875]}, {'id': 4, 'value': [0.583333333, 0.458333333, 0.0125]}, {'id': 5, 'value': [0.583333333, 0.958333333, 0.0875]}, {'id': 6, 'value': [0.083333333, 0.958333333, 0.0125]}, {'id': 7, 'value': [0.083333333, 0.458333333, 0.0875]}, {'id': 8, 'value': [0.25, 0.625, 0.1125]}, {'id': 9, 'value': [0.25, 0.125, 0.1875]}, {'id': 10, 'value': [0.75, 0.125, 0.1125]}, {'id': 11, 'value': [0.75, 0.625, 0.1875]}, {'id': 12, 'value': [0.25, 0.125, 0.1125]}, {'id': 13, 'value': [0.25, 0.625, 0.1875]}, {'id': 14, 'value': [0.75, 0.625, 0.1125]}, {'id': 15, 'value': [0.75, 0.125, 0.1875]}, {'id': 16, 'value': [0.916666667, 0.291666667, 0.2125]}, {'id': 17, 'value': [0.916666667, 0.791666667, 0.2875]}, {'id': 18, 'value': [0.416666667, 0.791666667, 0.2125]}, {'id': 19, 'value': [0.416666667, 0.291666667, 0.2875]}, {'id': 20, 'value': [0.916666667, 0.791666667, 0.2125]}, {'id': 21, 'value': [0.916666667, 0.291666667, 0.2875]}, {'id': 22, 'value': [0.416666667, 0.291666667, 0.2125]}, {'id': 23, 'value': [0.416666667, 0.791666667, 0.2875]}, {'id': 24, 'value': [0.583333333, 0.958333333, 0.3125]}, {'id': 25, 'value': [0.583333333, 0.458333333, 0.3875]}, {'id': 26, 'value': [0.083333333, 0.458333333, 0.3125]}, {'id': 27, 'value': [0.083333333, 0.958333333, 0.3875]}, {'id': 28, 'value': [0.583333333, 0.458333333, 0.3125]}, {'id': 29, 'value': [0.583333333, 0.958333333, 0.3875]}, {'id': 30, 'value': [0.083333333, 0.958333333, 0.3125]}, {'id': 31, 'value': [0.083333333, 0.458333333, 0.3875]}], 'units': 'crystal', 'cell': [[7.734, 0.0, 0.0], [3.867, 6.697840473, 0.0], [0.0, 0.0, 31.573922786]], 'constraints': [], 'labels': []}, 'lattice': {'a': 7.734, 'b': 7.734, 'c': 31.573922786, 'alpha': 90.0, 'beta': 90.0, 'gamma': 60.0, 'units': {'length': 'angstrom', 'angle': 'degree'}, 'type': 'TRI', 'vectors': {'a': [7.734, 0.0, 0.0], 'b': [3.867, 6.697840473, 0.0], 'c': [0.0, 0.0, 31.573922786], 'alat': 1, 'units': 'angstrom'}}, 'isNonPeriodic': False, '_id': '', 'metadata': {'boundaryConditions': {'type': 'pbc', 'offset': 0}, 'build': {'termination': 'Si_P6/mmm_4', 'configuration': {'type': 'SlabConfiguration', 'bulk': {'name': 'Si8', 'basis': {'elements': [{'id': 0, 'value': 'Si'}, {'id': 1, 'value': 'Si'}, {'id': 2, 'value': 'Si'}, {'id': 3, 'value': 'Si'}, {'id': 4, 'value': 'Si'}, {'id': 5, 'value': 'Si'}, {'id': 6, 'value': 'Si'}, {'id': 7, 'value': 'Si'}], 'coordinates': [{'id': 0, 'value': [0.5, 0.0, 0.0]}, {'id': 1, 'value': [0.25, 0.25, 0.75]}, {'id': 2, 'value': [0.5, 0.5, 0.5]}, {'id': 3, 'value': [0.25, 0.75, 0.25]}, {'id': 4, 'value': [0.0, 0.0, 0.5]}, {'id': 5, 'value': [0.75, 0.25, 0.25]}, {'id': 6, 'value': [0.0, 0.5, 0.0]}, {'id': 7, 'value': [0.75, 0.75, 0.75]}], 'units': 'crystal', 'cell': [[5.468763846, 0.0, 0.0], [0.0, 5.468763846, 0.0], [0.0, 0.0, 5.468763846]], 'constraints': [], 'labels': []}, 'lattice': {'a': 5.468763846, 'b': 5.468763846, 'c': 5.468763846, 'alpha': 90.0, 'beta': 90.0, 'gamma': 90.0, 'units': {'length': 'angstrom', 'angle': 'degree'}, 'type': 'TRI', 'vectors': {'a': [5.468763846, 0.0, 0.0], 'b': [0.0, 5.468763846, 0.0], 'c': [0.0, 0.0, 5.468763846], 'alat': 1, 'units': 'angstrom'}}, 'isNonPeriodic': False, '_id': '', 'metadata': {'boundaryConditions': {'type': 'pbc', 'offset': 0}}, 'isUpdated': True}, 'miller_indices': (1, 1, 1), 'thickness': 4, 'vacuum': 6, 'xy_supercell_matrix': [[1, 0], [0, 1]], 'use_conventional_cell': True, 'use_orthogonal_z': True, 'make_primitive': False}}}, 'isUpdated': True}, 'number_of_added_layers': 1, 'defect_type': 'ISLAND', 'condition': {'type': 'CylinderCoordinateCondition', 'center_position': [0.625, 0.5], 'radius': 0.25, 'min_z': 0.0, 'max_z': 1.0}}}}}

gh_test_material = Material(gh_test_material)
gh_test_slab = Material(gh_test_slab)
visualize([{"material": gh_test_slab, "title": "Original material"}, {"material": gh_test_material, "title": "Material with island defect"}])
visualize([{"material": gh_test_slab, "title": "Original material"}, {"material": gh_test_material, "title": "Material with island defect"}], rotation="-90x")


GridBox(children=(VBox(children=(Label(value='Si32 - Original material - rotation: 0x,0y,0z', layout=Layout(al…

GridBox(children=(VBox(children=(Label(value='Si32 - Original material - rotation: -90x', layout=Layout(align_…

In [25]:
from pymatgen.core.surface import SlabGenerator as PymatgenSlabGenerator
from mat3ra.made.tools.convert import to_pymatgen, from_pymatgen
clean_material = Material.create(Material.default_config)
generator = PymatgenSlabGenerator(
            initial_structure=to_pymatgen(clean_material),
            miller_index=(0,0,1),
            min_slab_size=3,
            min_vacuum_size=1,
            in_unit_planes=True,
            reorient_lattice=True,
            primitive=False
        )
raw_slabs = generator.get_slabs()
material_slabs = [Material(from_pymatgen(slab)) for slab in raw_slabs]
# material_slabs = [Material(from_pymatgen(slab.get_orthogonal_c_slab())) for slab in raw_slabs]
visualize(material_slabs)
visualize(material_slabs, rotation="-90x")

GridBox(children=(VBox(children=(Label(value='Si6 - Material - rotation: 0x,0y,0z', layout=Layout(align_self='…

GridBox(children=(VBox(children=(Label(value='Si6 - Material - rotation: -90x', layout=Layout(align_self='cent…